In [1]:
# load the data
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np


fname = "cmv.tar.bz2"
# url = "https://chenhaot.com/data/cmv/" + fname

# download if not exists
# if not os.path.isfile(fname):
#     f = BytesIO()
#     with request.urlopen(url) as resp, open(fname, 'wb') as f_disk:
#         data = resp.read()
#         f_disk.write(data)  # save to disk too
#         f.write(data)
#         f.seek(0)
# else:
f = open(fname, 'rb')


tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

all_train_fname = "all/train_period_data.jsonlist.bz2"
all_test_fname = "all/heldout_period_data.jsonlist.bz2"

pair_train_fname = "pair_task/train_pair_data.jsonlist.bz2"
pair_test_fname = "pair_task/train_pair_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)
all_train_bzlist = tar.extractfile(all_train_fname)
pair_train_bzlist = tar.extractfile(pair_train_fname)

# Deserialize the JSON list
original_posts_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]
all_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(all_train_bzlist)
]
pair_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(pair_train_bzlist)
]
test_bzlist = tar.extractfile(test_fname)
all_test_bzlist = tar.extractfile(all_test_fname)
pair_test_bzlist = tar.extractfile(pair_test_fname)

original_posts_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(test_bzlist)
]
all_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(all_test_bzlist)
]
pair_test = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(pair_test_bzlist)
]
f.close()

In [16]:
pair_train[0]

{'op_author': 'seanyowens',
 'op_text': 'I can\'t remember the topic that spurred this discussion, but a friend and I were debating whether man-made things were natural. He took the position that they are unnatural. \n\nHe cited this definition by Merriam-Webster:  existing in nature and not made or caused by people : coming from nature (http://www.merriam-webster.com/dictionary/natural) as his basis for the distinction for natural vs. unnatural.\n\nHowever, I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankind\'s capacity to create, problem-solve, analyze, rationalize, and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability, why is it that we would give the credit of these things solely to man, as opposed to nature? We are not separate from nature, thus, how can any of our actions or creations be unnatural? If we were somehow 

In [3]:
complete_key_set = []
for a in all_train:
    complete_key_set.extend(list(a.keys()))
complete_key_set = set(complete_key_set)
print(complete_key_set)

{'downs', 'clicked', 'ups', 'banned_by', 'num_reports', 'over_18', 'secure_media_embed', 'comments', 'media', 'edited', 'report_reasons', 'selftext_html', 'is_self', 'stickied', 'visited', 'media_embed', 'author_flair_css_class', 'permalink', 'created', 'suggested_sort', 'domain', 'subreddit', 'secure_media', 'name', 'archived', 'mod_reports', 'author_flair_text', 'url', 'hidden', 'selftext', 'likes', 'user_reports', 'thumbnail', 'title', 'distinguished', 'num_comments', 'approved_by', 'created_utc', 'gilded', 'subreddit_id', 'saved', 'id', 'link_flair_text', 'link_flair_css_class', 'author', 'score'}


In [4]:
def attch_to_parent(current, tree_dict, layer):
    # BFS search
    if current['parent_id'] in tree_dict:
        tree_dict[current['parent_id']][current['name']] = {}
        return None
    else:
        for k, v in tree_dict.items():
#             print(f"searching for {current['parent_id']} in {v.keys()}")
            attch_to_parent(current, v, layer + 1)

def print_tree(tree_dict, layer, orig, root):
    if tree_dict == {}:
        print('======================== end of subtree ========================')
        return
    else:
        for k, v in tree_dict.items():
            print(f'{" ".join(["*"] * layer)} {k}')
            if k != root:
                print(orig[k])
            print_tree(v, layer + 3, orig, root)

In [5]:
def show_comments(cmv_post):
    root = cmv_post['name']
    if 'comments' not in cmv_post:
        print('No comments found')
    else:
        print(len(cmv_post['comments']))
        tree_dict = {root: {}}
        for i in range(len(cmv_post['comments'])):
            current = cmv_post['comments'][i]
            attch_to_parent(current, tree_dict, 0)
        comments_dict = {c['name']: c['author'] + '\t' + c['body'] for c in cmv_post['comments']}
        print_tree(tree_dict, 0, comments_dict, root)


In [6]:
show_comments(all_train[0])

7
 t3_2rpvc7
* * * t1_cnibo8i
Kman17	I'd be a little bit cautious of removing the self-policing elements out of the sport.  A bench clearing brawl is usually retaliation for a headhunting pitcher.  

If you come down hard on retaliation, you then have to come down hard on the instigation as well... and before you know it, the sport itself is radically different.

Just look at the NBA.  The NBA, mostly after Jordan in the late 90's / early 00's, came down hard on player altercations.  This was for a variety of reasons mostly related to image / sportsmanship (and the racial double-standards put on mostly black players, but I digress) and to a lesser extent injuries.  

The occasional scuffles of the 80's / 90's became unheard of... and following that, the league came down hard on "chippy" fouls, and now they hand out double technicals any time anyone looks at anyone funny.  The end result is that reckless and sloppy charging to the basket, and relying on the superstar treatment to get to

In [7]:
# sanity check
for a in all_train[0]['comments']:
    print(f"{a['name']}\t{a['parent_id']}\t{a['replies']}")

t1_cnibo8i	t3_2rpvc7	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cnikfa7'], 'after': None, 'before': None}}
t1_cnikfa7	t1_cnibo8i	
t1_cni5tm6	t3_2rpvc7	
t1_cni6vol	t3_2rpvc7	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cni7f98'], 'after': None, 'before': None}}
t1_cni7f98	t1_cni6vol	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cniszwq'], 'after': None, 'before': None}}
t1_cniszwq	t1_cni7f98	{'kind': 'Listing', 'data': {'modhash': '', 'children': ['cnjaebu'], 'after': None, 'before': None}}
t1_cnjaebu	t1_cniszwq	


In [8]:
malleable_op = [post["name"] for post in original_posts_train if post["delta_label"]]
print(f'{len(original_posts_train)}, {len(malleable_op)}')

10743, 3191


In [9]:
for at in all_train:
    if at['name'] in malleable_op:
        show_comments(at)
#         print(at)
        break

30
 t3_2ro0ti
* * * t1_cnhpddf
Legomystrudel	May or may not have any sort of healing properties, but they smell really good.
* * * * * * t1_cnhpqan
GoldenTaint	Most do.  Some smell kinda funky.
======================== end of subtree ========================
* * * t1_cnhuxye
Bradm77	[This answer in /r/askscience does a pretty good job of describing essential oils.](http://www.reddit.com/r/askscience/comments/2pp57r/how_effective_are_essential_oils/cmytzdq) As the comment points out there are benefits to essential oils if they are used in the right way.  However, the current fad mixes the raw essential oils with other stuff that may or may not be good for you.  The raw essential oils are actually "a vital resource for drug discovery."
* * * * * * t1_cni1m79
GoldenTaint	&amp;#8710; While my view may not have shifted enough for me to avoid pissing my wife off, I have to say that you really brought home the bacon on this one and gave me some hope for the oils.  thank you for your response.

In [10]:
from IPython.display import Markdown


def show_post(cmv_post):
    md_format = "**{title}** \n\n {selftext}".format(**cmv_post)
    md_format = "\n".join(["> " + line for line in md_format.splitlines()])
    return Markdown(md_format)

In [11]:
show_post(all_train[200])

> **CMV: Their is no shame in pirating "The Interview" because of how poorly Sony handled the situation.** 
> 
>  Sony blundered the response to The Interview and then took a last minute turn because they saw a way to make a huge profit from it. While I don't think this was an intentional marketing gimmick, I also don't see a problem in pirating a movie they initially wanted to shelve indefinitely until they were shamed into releasing it. The company should be deeply embarrassed by how it acted during the situation, and I don't think they should be rewarded monetarily for the way they acted, even if they eventually did the "right" thing by releasing the film instead of indefinitely shelving it.
> 
> Even though Sony spent millions of dollars producing the movie, they handled the whole fiasco so ineptly that the President, the media, and the internet basically shamed them into releasing it. There should be no shame in our part for downloading a movie if one doesn't want to support the studio for their mismanagement. 
> 
> Edit: I feel pretty stupid that I obviously used the wrong use of "there" in the title.
> 
> _____
> 
> &gt; *Hello, users of CMV! This is a footnote from your moderators. We'd just like to remind you of a couple of things. Firstly, please remember to* ***[read through our rules](http://www.reddit.com/r/changemyview/wiki/rules)***. *If you see a comment that has broken one, it is more effective to report it than downvote it. Speaking of which,* ***[downvotes don't change views](http://www.reddit.com/r/changemyview/wiki/guidelines#wiki_upvoting.2Fdownvoting)****! If you are thinking about submitting a CMV yourself, please have a look through our* ***[popular topics wiki](http://www.reddit.com/r/changemyview/wiki/populartopics)*** *first. Any questions or concerns? Feel free to* ***[message us](http://www.reddit.com/message/compose?to=/r/changemyview)***. *Happy CMVing!*

In [12]:
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
            ]
    return "\n".join(lines)

In [13]:
show_post(dict(title=all_train[200]['title'],
               selftext=cleanup(all_train[200]['selftext'])))

> **CMV: Their is no shame in pirating "The Interview" because of how poorly Sony handled the situation.** 
> 
>  Sony blundered the response to The Interview and then took a last minute turn because they saw a way to make a huge profit from it. While I don't think this was an intentional marketing gimmick, I also don't see a problem in pirating a movie they initially wanted to shelve indefinitely until they were shamed into releasing it. The company should be deeply embarrassed by how it acted during the situation, and I don't think they should be rewarded monetarily for the way they acted, even if they eventually did the "right" thing by releasing the film instead of indefinitely shelving it.
> 
> Even though Sony spent millions of dollars producing the movie, they handled the whole fiasco so ineptly that the President, the media, and the internet basically shamed them into releasing it. There should be no shame in our part for downloading a movie if one doesn't want to support the studio for their mismanagement. 
> 
> 

In [14]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

In [22]:
pair_train[0]['positive']['comments'][0]['body']

'Look at the definition you provided, if we remove the exclusion of things which humans create:\n\n&gt; existing in nature ~~and not made or caused by people~~\n\nSo essentially, by this definition, "natural things" are "things that exist," which is frankly rather meaningless. If one wanted to discuss the results of human activity we would then have to make up a new word which could be redefined by the same argument. \n\nThe whole point of the word is to exclude human activity. If you remove that aspect, it simply ceases to have meaning.'

In [23]:
# prepare the data in Python list form

text_train = [
    cleanup(tr['positive']['comments'][0]['body'])
    for tr
    in pair_train
]

text_test = [
    cleanup(te['positive']['comments'][0]['body'])
    for te
    in pair_test
]

In [24]:
text_train[:3]

['Look at the definition you provided, if we remove the exclusion of things which humans create:\n\n\nSo essentially, by this definition, "natural things" are "things that exist," which is frankly rather meaningless. If one wanted to discuss the results of human activity we would then have to make up a new word which could be redefined by the same argument. \n\nThe whole point of the word is to exclude human activity. If you remove that aspect, it simply ceases to have meaning.',
 '[This answer in /r/askscience does a pretty good job of describing essential oils.](http://www.reddit.com/r/askscience/comments/2pp57r/how_effective_are_essential_oils/cmytzdq) As the comment points out there are benefits to essential oils if they are used in the right way.  However, the current fad mixes the raw essential oils with other stuff that may or may not be good for you.  The raw essential oils are actually "a vital resource for drug discovery."',
 'I guess I look at it this way- there have absolut

In [26]:
vect = TfidfVectorizer(use_idf=False, norm='l1')
X_train = vect.fit_transform(text_train)

In [28]:
print(X_train[:5])

  (0, 16196)	0.012345679012345678
  (0, 2942)	0.012345679012345678
  (0, 26677)	0.07407407407407407
  (0, 7675)	0.024691358024691357
  (0, 29803)	0.024691358024691357
  (0, 21266)	0.012345679012345678
  (0, 13590)	0.037037037037037035
  (0, 29141)	0.024691358024691357
  (0, 22436)	0.024691358024691357
  (0, 10141)	0.012345679012345678
  (0, 18687)	0.037037037037037035
  (0, 26796)	0.037037037037037035
  (0, 29284)	0.037037037037037035
  (0, 13363)	0.012345679012345678
  (0, 6968)	0.012345679012345678
  (0, 24722)	0.012345679012345678
  (0, 9934)	0.012345679012345678
  (0, 4606)	0.024691358024691357
  (0, 26815)	0.012345679012345678
  (0, 17961)	0.012345679012345678
  (0, 2640)	0.012345679012345678
  (0, 26670)	0.024691358024691357
  (0, 10201)	0.012345679012345678
  (0, 14789)	0.024691358024691357
  (0, 11385)	0.012345679012345678
  :	:
  (4, 1848)	0.0017123287671232876
  (4, 18145)	0.0017123287671232876
  (4, 6644)	0.0017123287671232876
  (4, 24198)	0.003424657534246575
  (4, 2644)	0.

In [ ]:
lr = LogisticRegressionCV(Cs=10, class_weight='balanced',
                          scoring='roc_auc', solver='sag',
                          tol=0.001, max_iter=500,
                          random_state=0)

In [ ]:
is_malleable = np.array([post["delta_label"] for post in original_posts_train])

lr.fit(X_train, is_malleable)

In [ ]:
def top_features(coef, words, n=15):
    order = np.argsort(-coef)
    for i_hi, i_lo in zip(order[:n], order[:-n - 1:-1]):
        row_vals = coef[i_hi], words[i_hi], coef[i_lo], words[i_lo]
        print("[{:>4.1f}] {:<20}[{:>4.1f}] {:<20}".format(*row_vals))

In [ ]:
top_features(lr.coef_.flatten(), vect.get_feature_names_out())